# データ整形
df：  
yt → 観測値  
nan_flags → 欠損値なしは0、ありは1  
その他、使用する変数を格納  

## blsallfood.csvの読み込み

In [1]:
import pandas as pd

df = pd.read_csv(r"C:\asp\state_space_model\blsallfood.csv")

In [2]:
df

,yt
0,1720
1,1702
2,1707
3,1708
4,1727
...,...
151,1829
152,1835
153,1782
154,1736


In [3]:
any_na = df['yt'].isna().any()
if any_na:
    print("`yt` 列には欠損値が存在します。")
else:
    print("`yt` 列には欠損値は存在しません。")

`yt` 列には欠損値は存在しません。


## dfの整形

In [4]:
df['nan_flags'] = 0

df

,yt,nan_flags
0,1720,0
1,1702,0
2,1707,0
3,1708,0
4,1727,0
...,...,...
151,1829,0
152,1835,0
153,1782,0
154,1736,0


## dfの各列をseriesに変換

In [5]:
series_y = df['yt']
series_nan_flags = df['nan_flags']

In [6]:
series_y

0      1720
1      1702
2      1707
3      1708
4      1727
       ... 
151    1829
152    1835
153    1782
154    1736
155    1706
Name: yt, Length: 156, dtype: int64

In [7]:
series_nan_flags

0      0
1      0
2      0
3      0
4      0
      ..
151    0
152    0
153    0
154    0
155    0
Name: nan_flags, Length: 156, dtype: int64

# 対数尤度の計算

In [ ]:
import numpy as np

In [43]:
def KF_func(t, series_y, F, G, Q, H, R, I, x_t_1_t_1, V_t_1_t_1, series_nan_flags, sta, end, end_pred):

    # F_t, G_t, Q_t, H_t, R_t の定義(時刻tについての行列)
    F_t = F.copy()
    G_t = G.copy()
    Q_t = Q.copy()
    H_t = H.copy()
    R_t = R.copy()

    # 1期先予測
    x_t_t_1 = F_t @ x_t_1_t_1
    V_t_t_1 = F_t @ V_t_1_t_1 @ F_t.T + G_t @ Q_t @ G_t.T

    # フィルタ
    ## 欠損値なし
    if series_nan_flags[t] == 0:
        d_t_t_1 = H_t @ V_t_t_1 @ H_t.T + R_t  # 対数尤度の計算で使用される
        K_t = V_t_t_1 @ H_t.T @ np.linalg.inv(d_t_t_1)
        x_t_t = x_t_t_1 + K_t @ ((series_y[t] - series_y[sta:end].mean()) - H_t @ x_t_t_1)
        V_t_t = (I - K_t @ H_t) @ V_t_t_1
    ## 欠損値あり
    else:
        d_t_t_1 = None  # 対数尤度の計算で使用される（欠損値がある場合についてはスキップしなければならない）
        K_t = None
        x_t_t = x_t_t_1
        V_t_t = V_t_t_1
    ## 予測誤差の算出(予測時のみ→d_t_j_t ** (1/2)、それ以外はNoneを返す)
    if end <= t < end_pred:
        d_t_j_t = H_t @ V_t_t_1 @ H_t.T + R_t
        root_d_t_j_t = d_t_j_t ** (1/2)
    else:
        d_t_j_t = None
        root_d_t_j_t = None

    # 予測値yの算出
    ## y_t_t_1
    y_t_t_1 = H_t @ x_t_t_1 + series_y[sta:end].mean()

    ## y_t_t
    y_t_t = H_t @ x_t_t + series_y[sta:end].mean()

    return x_t_t_1, V_t_t_1, d_t_t_1, K_t, x_t_t, V_t_t, y_t_t_1, y_t_t, root_d_t_j_t

In [ ]:
def loglike_func(series_y, series_nan_flags):
    # 行列の定義(F, G, Q, H, I)
    ##  F：推移行列(15 * 15)
    F = np.zeros([15, 15])
    F[0, :] = [
        1.13164633, -0.13384263, -0.25397419, 0.02039985,0.03500777,
        0.05990097, -0.17683803, 0.08439510, 0.10236195, -0.12517702,
        0.10852823, 0.64089953, -0.74428281, 0.04803834, 0.15332373
        ]
    for i in range(1, 15):
        F[i, i - 1] = 1

    ## G：(15 * 1)
    G = np.zeros([15, 1])
    G[0, 0] = 1

    ## Q：システムノイズの分散(1 * 1)
    Q = np.identity(1) * 422.747668985944

    ## H：観測行列(1 * 15)
    H = np.zeros([1, 15])
    H[0, 0] = 1

    ## I：単位行列(15 * 15)
    I = np.identity(15)

    ## R：観測ノイズの分散共分散行列(1*1) → 尤度を計算する際には R = 1 で設定される。
    R = np.identity(1)

    # 初期値の定義
    ## x_0_0：初期状態(15 * 1)
    x_0_0 = np.zeros([15, 1])

    ## V_0_0：初期分散共分散行列(15 * 15)
    V_0_0 = np.identity(15)
    V_0_0 = V_0_0 * 1e5

    # staとendの定義
    sta = 0
    end = 120  ### 使用するデータ数を定義
    end_pred = end

    # カルマンフィルタの計算
    ## トレーニングデータの取得
    y_train = series_y[sta:end]
    nan_flags_train = series_nan_flags[sta:end]

    ## 予測データの作成
    y_pred = pd.Series(series_y[end - 1], index = range(end, end_pred))
    nan_flags_pred = pd.Series(1, index = range(end, end_pred))

    ## トレーニングデータと予測データのマージ：カルマンフィルタの計算で使用される
    y_t = pd.concat([y_train, y_pred])
    nan_flags_t = pd.concat([nan_flags_train, nan_flags_pred])

    ## t_max：データ長の定義
    t_max = end_pred - sta

    ## カルマンフィルタの結果を格納するリストの定義
    list_x_t_t_1 =[]
    list_V_t_t_1 = []
    list_d_t_t_1_tilde = [] # R=1で計算しているため
    list_K_t = []
    list_x_t_t = []
    list_V_t_t = []
    list_y_t_t_1 = []
    list_y_t_t = []
    list_root_d_t_j_t = []

    ## カルマンフィルタの計算
    for t in range(t_max):
        ### KF_funcの利用
        if t == 0:
            x_t_t_1, V_t_t_1, d_t_t_1_tilde, K_t, x_t_t, V_t_t, y_t_t_1, y_t_t, root_d_t_j_t = KF_func(t, y_t, F, G, Q, H, R, I, x_0_0, V_0_0, nan_flags_t, sta, end, end_pred)
        else:
            x_t_1_t_1 = list_x_t_t[t-1]
            V_t_1_t_1 = list_V_t_t[t-1]
            x_t_t_1, V_t_t_1, d_t_t_1_tilde, K_t, x_t_t, V_t_t, y_t_t_1, y_t_t, root_d_t_j_t = KF_func(t, y_t, F, G, Q, H, R, I, x_t_1_t_1, V_t_1_t_1, nan_flags_t, sta, end, end_pred)
        ### リストに計算結果を挿入
        list_x_t_t_1.append(x_t_t_1)
        list_V_t_t_1.append(V_t_t_1)
        list_d_t_t_1_tilde.append(d_t_t_1_tilde)
        list_K_t.append(K_t)
        list_x_t_t.append(x_t_t)
        list_V_t_t.append(V_t_t)
        list_y_t_t_1.append(y_t_t_1)
        list_y_t_t.append(y_t_t)
        list_root_d_t_j_t.append(root_d_t_j_t)

    # 対数尤度の計算
    ## sigma_hat_squared の計算
    sigma_hat_squared = 0
    T = 0

    for t in range(t_max):
        if list_d_t_t_1_tilde[t] is not None:
            sigma_hat_squared += (y_t[t] - list_y_t_t_1[t][0]) ** 2 / list_d_t_t_1_tilde[t][0][0]
            T += 1

    sigma_hat_squared = sigma_hat_squared / T

    ## 対数尤度の計算
    ### sigma_ln_d_t_t_1_tilde の計算
    sigma_ln_d_t_t_1_tilde = 0

    for t in range(t_max):
        if list_d_t_t_1_tilde[t] is not None:
            sigma_ln_d_t_t_1_tilde += np.log(list_d_t_t_1_tilde[t][0][0])

    ### l_theta_asterisk の計算
    l_theta_asterisk = -0.5 * (T * np.log(2 * np.pi * sigma_hat_squared) + sigma_ln_d_t_t_1_tilde + T)

    # 返り値の定義
    return list_x_t_t_1, list_V_t_t_1, list_d_t_t_1_tilde, list_K_t, list_x_t_t, list_V_t_t, list_y_t_t_1, list_y_t_t, list_root_d_t_j_t, F, G, Q, H, I, R, x_0_0, V_0_0, sta, end, end_pred, l_theta_asterisk

In [106]:
list_x_t_t_1, list_V_t_t_1, list_d_t_t_1_tilde, list_K_t, list_x_t_t, list_V_t_t, list_y_t_t_1, list_y_t_t, list_root_d_t_j_t, F, G, Q, H, I, R, x_0_0, V_0_0, sta, end, end_pred, l_theta_asterisk = loglike_func(series_y, series_nan_flags)

In [ ]:
l_theta_asterisk  # 対数尤度であり、最大を取るθ*を求めることが目的

array([-516.98515652])

## 結果の表示

In [109]:
print("対数尤度：", float(l_theta_asterisk[0]))

対数尤度： -516.9851565176928
